In [ ]:
# RebalanceAI: AI-Powered Investment Portfolio Analyzer
# Hackathon Submission - Team [Your Team #]

"""
This notebook implements RebalanceAI, an intelligent portfolio recommendation system
that combines quantitative finance metrics with real-time sentiment analysis using LangChain.
"""

# ============================================================================
# PART 1: SETUP AND IMPORTS
# ============================================================================

# Install required packages (run in terminal or uncomment):
# pip install langchain langchain-openai yfinance pandas numpy matplotlib plotly newsapi-python praw python-dotenv

import os
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Any
from google.colab import userdata


# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# Optional: For sentiment analysis
from newsapi import NewsApiClient
# import praw  # For Reddit

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Configure API keys
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["NEWSAPI_KEY"] = userdata.get('NEWSAPI_KEY')

# ============================================================================
# PART 2: DATA INGESTION TOOLS
# ============================================================================

def fetch_stock_data(ticker: str, period: str = "1y") -> Dict[str, Any]:
    """
    Fetch historical stock data from Yahoo Finance.

    Args:
        ticker: Stock symbol (e.g., 'AAPL')
        period: Time period ('1mo', '3mo', '6mo', '1y', '2y', '5y')

    Returns:
        Dictionary with stock data and metadata
    """
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)

        if df.empty:
            return {'success': False, 'error': f'No data found for {ticker}'}

        # Calculate daily returns
        df['Returns'] = df['Close'].pct_change()

        # Get company info
        info = stock.info

        return {
            'ticker': ticker,
            'data': df,
            'current_price': float(df['Close'].iloc[-1]),
            'company_name': info.get('longName', ticker),
            'sector': info.get('sector', 'Unknown'),
            'market_cap': info.get('marketCap', 0),
            'success': True
        }
    except Exception as e:
        return {'success': False, 'error': str(e)}

def fetch_multiple_stocks(tickers: List[str], period: str = "1y") -> Dict[str, Any]:
    """Fetch data for multiple stocks at once."""
    results = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}...")
        results[ticker] = fetch_stock_data(ticker, period)
    return results

# ============================================================================
# PART 3: QUANTITATIVE ANALYSIS TOOLS
# ============================================================================

def calculate_metrics(ticker_data: Dict[str, Any]) -> Dict[str, float]:
    """
    Calculate key financial metrics for a stock.

    Returns:
        Dictionary with volatility, Sharpe ratio, beta, max drawdown, etc.
    """
    if not ticker_data.get('success'):
        return {'error': ticker_data.get('error')}

    df = ticker_data['data']
    returns = df['Returns'].dropna()

    # Volatility (annualized)
    volatility = returns.std() * np.sqrt(252)

    # Average annual return
    avg_return = returns.mean() * 252

    # Sharpe Ratio (assuming 4% risk-free rate)
    risk_free_rate = 0.04
    sharpe_ratio = (avg_return - risk_free_rate) / volatility if volatility > 0 else 0

    # Max Drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()

    # Moving averages
    df['MA_50'] = df['Close'].rolling(window=50).mean()
    df['MA_200'] = df['Close'].rolling(window=200).mean()

    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    current_rsi = rsi.iloc[-1] if len(rsi) > 0 else 50

    return {
        'volatility': round(volatility, 4),
        'sharpe_ratio': round(sharpe_ratio, 4),
        'max_drawdown': round(max_drawdown, 4),
        'avg_annual_return': round(avg_return, 4),
        'current_rsi': round(current_rsi, 2),
        'ma_50': round(df['MA_50'].iloc[-1], 2) if len(df['MA_50']) > 0 else 0,
        'ma_200': round(df['MA_200'].iloc[-1], 2) if len(df['MA_200']) > 0 else 0
    }

def calculate_portfolio_metrics(stocks_data: Dict[str, Dict], weights: Dict[str, float]) -> Dict[str, float]:
    """
    Calculate portfolio-level metrics given stocks and their weights.

    Args:
        stocks_data: Dictionary of ticker -> stock data
        weights: Dictionary of ticker -> weight (must sum to 1.0)

    Returns:
        Portfolio volatility, expected return, Sharpe ratio
    """
    # Extract returns for all stocks
    returns_df = pd.DataFrame()
    for ticker, data in stocks_data.items():
        if data.get('success'):
            returns_df[ticker] = data['data']['Returns']

    returns_df = returns_df.dropna()

    # Portfolio return (weighted average)
    weight_array = np.array([weights.get(t, 0) for t in returns_df.columns])
    portfolio_return = (returns_df.mean() @ weight_array) * 252

    # Portfolio volatility
    cov_matrix = returns_df.cov() * 252
    portfolio_variance = weight_array @ cov_matrix @ weight_array
    portfolio_volatility = np.sqrt(portfolio_variance)

    # Portfolio Sharpe ratio
    risk_free_rate = 0.04
    portfolio_sharpe = (portfolio_return - risk_free_rate) / portfolio_volatility

    return {
        'portfolio_return': round(portfolio_return, 4),
        'portfolio_volatility': round(portfolio_volatility, 4),
        'portfolio_sharpe': round(portfolio_sharpe, 4)
    }

# ============================================================================
# PART 4: SENTIMENT ANALYSIS TOOLS
# ============================================================================

def fetch_news(ticker: str, num_articles: int = 5) -> List[Dict[str, str]]:
    """
    Fetch recent news articles for a stock using NewsAPI.

    Args:
        ticker: Stock symbol
        num_articles: Number of articles to fetch

    Returns:
        List of articles with title, description, and source
    """
    try:
        if not NEWSAPI_KEY:
            return [{'error': 'NewsAPI key not configured'}]

        newsapi = NewsApiClient(api_key=NEWSAPI_KEY)

        # Get company name for better search
        stock = yf.Ticker(ticker)
        company_name = stock.info.get('longName', ticker)

        # Fetch news from last 7 days
        from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

        articles = newsapi.get_everything(
            q=company_name,
            from_param=from_date,
            language='en',
            sort_by='relevancy',
            page_size=num_articles
        )

        return [{
            'title': article['title'],
            'description': article['description'],
            'source': article['source']['name'],
            'published_at': article['publishedAt']
        } for article in articles.get('articles', [])]

    except Exception as e:
        return [{'error': str(e)}]

def analyze_sentiment_with_llm(text: str, llm) -> Dict[str, Any]:
    """
    Use LLM to analyze sentiment of news/text.

    Args:
        text: Text to analyze
        llm: LangChain LLM instance

    Returns:
        Sentiment score (-1 to 1) and analysis
    """
    prompt = f"""Analyze the following financial news and provide:
1. A sentiment score from -1 (very negative) to 1 (very positive)
2. Key themes or topics
3. Brief investment implication

News: {text}

Respond in this format:
Score: [number between -1 and 1]
Themes: [key themes]
Implication: [brief implication]
"""

    try:
        response = llm.invoke(prompt)
        content = response.content

        # Parse score from response
        score_line = [line for line in content.split('\n') if 'Score:' in line]
        score = 0.0
        if score_line:
            try:
                score = float(score_line[0].split(':')[1].strip())
            except:
                score = 0.0

        return {
            'score': score,
            'analysis': content
        }
    except Exception as e:
        return {'score': 0.0, 'analysis': f'Error: {str(e)}'}

def aggregate_sentiment(ticker: str, llm) -> Dict[str, Any]:
    """
    Fetch news and compute aggregate sentiment for a stock.

    Returns:
        Average sentiment score and summary
    """
    articles = fetch_news(ticker, num_articles=5)

    if not articles or 'error' in articles[0]:
        return {
            'ticker': ticker,
            'avg_sentiment': 0.0,
            'summary': 'No news data available'
        }

    sentiments = []
    analyses = []

    for article in articles[:3]:  # Analyze top 3 articles
        text = f"{article['title']}. {article.get('description', '')}"
        sentiment = analyze_sentiment_with_llm(text, llm)
        sentiments.append(sentiment['score'])
        analyses.append(sentiment['analysis'])

    avg_sentiment = np.mean(sentiments) if sentiments else 0.0

    return {
        'ticker': ticker,
        'avg_sentiment': round(avg_sentiment, 3),
        'num_articles': len(articles),
        'summary': f"Analyzed {len(sentiments)} articles. Average sentiment: {avg_sentiment:.3f}"
    }

# ============================================================================
# PART 5: RISK ASSESSMENT TOOLS
# ============================================================================

def assess_risk_profile(metrics: Dict[str, float]) -> str:
    """
    Determine risk category based on volatility and other metrics.

    Returns:
        'Conservative', 'Moderate', or 'Aggressive'
    """
    volatility = metrics.get('volatility', 0)
    sharpe = metrics.get('sharpe_ratio', 0)

    if volatility < 0.15:
        return 'Conservative'
    elif volatility < 0.25:
        return 'Moderate'
    else:
        return 'Aggressive'

def filter_stocks_by_risk(stocks_metrics: Dict[str, Dict],
                         target_risk: str) -> List[str]:
    """
    Filter stocks that match a target risk profile.

    Args:
        stocks_metrics: Dictionary of ticker -> metrics
        target_risk: 'Conservative', 'Moderate', or 'Aggressive'

    Returns:
        List of tickers matching the risk profile
    """
    matching_stocks = []

    for ticker, metrics in stocks_metrics.items():
        if 'error' not in metrics:
            risk_profile = assess_risk_profile(metrics)
            if risk_profile == target_risk:
                matching_stocks.append(ticker)

    return matching_stocks

# ============================================================================
# PART 6: PORTFOLIO OPTIMIZATION
# ============================================================================

def optimize_portfolio_simple(stocks_data: Dict[str, Dict],
                              risk_profile: str,
                              sentiment_scores: Dict[str, float]) -> Dict[str, Any]:
    """
    Generate portfolio allocation using simple optimization logic.

    Combines:
    - Risk-adjusted returns (Sharpe ratio)
    - Sentiment scores
    - Risk profile constraints

    Args:
        stocks_data: Dictionary of stock data
        risk_profile: Target risk level
        sentiment_scores: Dictionary of ticker -> sentiment score

    Returns:
        Recommended allocation and reasoning
    """
    # Calculate scores for each stock
    stock_scores = {}

    for ticker, data in stocks_data.items():
        if not data.get('success'):
            continue

        metrics = calculate_metrics(data)
        if 'error' in metrics:
            continue

        # Base score from Sharpe ratio
        sharpe_score = metrics['sharpe_ratio']

        # Sentiment adjustment (-0.2 to +0.2)
        sentiment = sentiment_scores.get(ticker, 0)
        sentiment_adjustment = sentiment * 0.2

        # Risk adjustment based on profile
        volatility = metrics['volatility']
        if risk_profile == 'Conservative' and volatility > 0.25:
            risk_penalty = -0.5
        elif risk_profile == 'Aggressive' and volatility < 0.15:
            risk_penalty = -0.3
        else:
            risk_penalty = 0

        # Combined score
        total_score = sharpe_score + sentiment_adjustment + risk_penalty
        stock_scores[ticker] = max(total_score, 0)  # No negative scores

    # Normalize scores to weights
    total_score = sum(stock_scores.values())
    if total_score == 0:
        # Equal weight if all scores are zero
        weights = {t: 1.0/len(stock_scores) for t in stock_scores}
    else:
        weights = {t: score/total_score for t, score in stock_scores.items()}

    # Calculate portfolio metrics
    portfolio_metrics = calculate_portfolio_metrics(stocks_data, weights)

    return {
        'weights': weights,
        'portfolio_metrics': portfolio_metrics,
        'stock_scores': stock_scores,
        'reasoning': generate_allocation_reasoning(weights, stock_scores,
                                                   sentiment_scores, risk_profile)
    }

def generate_allocation_reasoning(weights: Dict[str, float],
                                 scores: Dict[str, float],
                                 sentiments: Dict[str, float],
                                 risk_profile: str) -> str:
    """Generate natural language explanation for the allocation."""

    sorted_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)

    reasoning = f"For a {risk_profile} risk profile, I recommend:\n\n"

    for ticker, weight in sorted_weights:
        percentage = weight * 100
        sentiment = sentiments.get(ticker, 0)
        sentiment_label = "positive" if sentiment > 0.1 else "negative" if sentiment < -0.1 else "neutral"

        reasoning += f"- {ticker}: {percentage:.1f}% (Score: {scores[ticker]:.2f}, Sentiment: {sentiment_label})\n"

    reasoning += f"\nThis allocation balances risk-adjusted returns with current market sentiment."

    return reasoning

# ============================================================================
# PART 7: LANGCHAIN AGENT SETUP
# ============================================================================

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Define tools for the agent
def tool_fetch_stock_data(ticker: str) -> str:
    """Fetches historical stock data for a given ticker symbol."""
    result = fetch_stock_data(ticker)
    if result['success']:
        return f"Successfully fetched data for {ticker}. Current price: ${result['current_price']:.2f}"
    return f"Error fetching {ticker}: {result['error']}"

def tool_calculate_metrics(ticker: str) -> str:
    """Calculates financial metrics like volatility, Sharpe ratio for a stock."""
    data = fetch_stock_data(ticker)
    metrics = calculate_metrics(data)
    if 'error' not in metrics:
        return f"{ticker} Metrics: Volatility={metrics['volatility']}, Sharpe={metrics['sharpe_ratio']}, Max Drawdown={metrics['max_drawdown']}"
    return f"Error: {metrics['error']}"

def tool_analyze_sentiment(ticker: str) -> str:
    """Analyzes news sentiment for a stock."""
    sentiment_data = aggregate_sentiment(ticker, llm)
    return f"{ticker} Sentiment: {sentiment_data['summary']}"

def tool_optimize_portfolio(tickers: str, risk_profile: str) -> str:
    """
    Optimizes portfolio allocation for given tickers and risk profile.
    tickers should be comma-separated (e.g., 'AAPL,MSFT,GOOGL')
    risk_profile should be 'Conservative', 'Moderate', or 'Aggressive'
    """
    ticker_list = [t.strip() for t in tickers.split(',')]

    # Fetch data
    stocks_data = fetch_multiple_stocks(ticker_list)

    # Get sentiments
    sentiment_scores = {}
    for ticker in ticker_list:
        sentiment_data = aggregate_sentiment(ticker, llm)
        sentiment_scores[ticker] = sentiment_data['avg_sentiment']

    # Optimize
    result = optimize_portfolio_simple(stocks_data, risk_profile, sentiment_scores)

    output = f"Portfolio Optimization Results:\n\n"
    output += result['reasoning']
    output += f"\n\nPortfolio Metrics:\n"
    output += f"Expected Return: {result['portfolio_metrics']['portfolio_return']*100:.2f}%\n"
    output += f"Volatility: {result['portfolio_metrics']['portfolio_volatility']*100:.2f}%\n"
    output += f"Sharpe Ratio: {result['portfolio_metrics']['portfolio_sharpe']:.2f}"

    return output

# Create tool objects
tools = [
    Tool(
        name="fetch_stock_data",
        func=tool_fetch_stock_data,
        description="Fetches historical stock data for a ticker. Input: ticker symbol (e.g., 'AAPL')"
    ),
    Tool(
        name="calculate_metrics",
        func=tool_calculate_metrics,
        description="Calculates financial metrics (volatility, Sharpe ratio, etc.) for a stock. Input: ticker symbol"
    ),
    Tool(
        name="analyze_sentiment",
        func=tool_analyze_sentiment,
        description="Analyzes news sentiment for a stock. Input: ticker symbol"
    ),
    Tool(
        name="optimize_portfolio",
        func=tool_optimize_portfolio,
        description="Optimizes portfolio allocation. Input: comma-separated tickers and risk profile (e.g., 'AAPL,MSFT,GOOGL|Moderate')"
    )
]

# Define agent prompt
agent_prompt = PromptTemplate.from_template("""You are RebalanceAI, an intelligent investment portfolio advisor.

You help users analyze stocks, assess risk, and optimize portfolio allocations by combining:
1. Quantitative financial metrics (volatility, Sharpe ratio, etc.)
2. Real-time market sentiment from news
3. Risk profile preferences

Available tools:
{tools}

Tool names: {tool_names}

When answering investment questions:
- Always gather both quantitative metrics AND sentiment data
- Explain your reasoning clearly
- Acknowledge risks and uncertainties
- Remind users this is educational, not professional financial advice

Question: {input}

Thought: {agent_scratchpad}
""")

# Create the agent
agent = create_react_agent(llm, tools, agent_prompt)

# Create agent executor with memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

# ============================================================================
# PART 8: TEST THE AGENT
# ============================================================================

print("=" * 80)
print("RebalanceAI - Ready!")
print("=" * 80)

# Test Query 1: Single stock analysis
print("\n\nTest Query 1: Analyze AAPL with metrics and sentiment")
response1 = agent_executor.invoke({
    "input": "Analyze AAPL. Show me key financial metrics and current sentiment."
})
print("\nResponse:", response1['output'])

# Test Query 2: Compare two stocks
print("\n\n" + "="*80)
print("Test Query 2: Compare TSLA vs NVDA")
response2 = agent_executor.invoke({
    "input": "Compare TSLA vs NVDA on volatility, Sharpe ratio, and sentiment. Which is better for moderate risk?"
})
print("\nResponse:", response2['output'])

# Test Query 3: Portfolio optimization
print("\n\n" + "="*80)
print("Test Query 3: Optimize portfolio")
response3 = agent_executor.invoke({
    "input": "Create a $10,000 portfolio with AAPL, MSFT, GOOGL for a moderate risk investor. Show allocation and reasoning."
})
print("\nResponse:", response3['output'])

# ============================================================================
# PART 9: VISUALIZATION (Optional Enhancement)
# ============================================================================

def visualize_portfolio(weights: Dict[str, float], stocks_data: Dict[str, Dict]):
    """Create visualizations for portfolio analysis."""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Portfolio Analysis Dashboard', fontsize=16, fontweight='bold')

    # 1. Allocation Pie Chart
    tickers = list(weights.keys())
    weight_values = [weights[t]*100 for t in tickers]

    axes[0, 0].pie(weight_values, labels=tickers, autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Portfolio Allocation')

    # 2. Risk-Return Scatter
    returns = []
    volatilities = []
    for ticker in tickers:
        metrics = calculate_metrics(stocks_data[ticker])
        returns.append(metrics['avg_annual_return'] * 100)
        volatilities.append(metrics['volatility'] * 100)

    axes[0, 1].scatter(volatilities, returns, s=200, alpha=0.6)
    for i, ticker in enumerate(tickers):
        axes[0, 1].annotate(ticker, (volatilities[i], returns[i]))
    axes[0, 1].set_xlabel('Volatility (%)')
    axes[0, 1].set_ylabel('Annual Return (%)')
    axes[0, 1].set_title('Risk-Return Profile')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Sharpe Ratio Comparison
    sharpe_ratios = []
    for ticker in tickers:
        metrics = calculate_metrics(stocks_data[ticker])
        sharpe_ratios.append(metrics['sharpe_ratio'])

    axes[1, 0].bar(tickers, sharpe_ratios, color='steelblue')
    axes[1, 0].set_ylabel('Sharpe Ratio')
    axes[1, 0].set_title('Risk-Adjusted Performance')
    axes[1, 0].grid(True, alpha=0.3, axis='y')

    # 4. Historical Price Trends (normalized)
    for ticker in tickers:
        df = stocks_data[ticker]['data']
        normalized_price = (df['Close'] / df['Close'].iloc[0]) * 100
        axes[1, 1].plot(df.index, normalized_price, label=ticker, linewidth=2)

    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Normalized Price (Base=100)')
    axes[1, 1].set_title('Historical Price Performance')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

print("\n\n" + "="*80)
print("RebalanceAI Setup Complete!")
print("="*80)
print("\nYou can now:")
print("1. Test the agent with different queries")
print("2. Add more tools and capabilities")
print("3. Create visualizations with visualize_portfolio()")
print("4. Enhance sentiment analysis with Reddit/Twitter data")
print("\nIMPORTANT: This is educational software. Always consult professional")
print("financial advisors before making investment decisions.")

/tmp/ipython-input-1611354678.py:556: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


RebalanceAI - Ready!


Test Query 1: Analyze AAPL with metrics and sentiment


> Entering new AgentExecutor chain...
To analyze AAPL (Apple Inc.), I will fetch the historical stock data, calculate key financial metrics, and analyze the current market sentiment.

First, let's fetch the historical stock data using the `fetch_stock_data('AAPL')` tool. This will provide us with a comprehensive view of the stock's performance over time.

Next, we'll calculate key financial metrics using the `calculate_metrics('AAPL')` tool. This will give us insights into the stock's volatility, Sharpe ratio, and other important indicators. Volatility measures the degree of variation in the trading prices over time, indicating the level of risk. The Sharpe ratio, on the other hand, measures the performance of an investment compared to a risk-free asset, after adjusting for its risk. It is a useful tool to understand the return of an investment compared to its risk.

Finally, we'll analyze the current market

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: - 'TSLA'- 'NVDA'NEXT, I WILL CALCULATE THE FINANCIAL METRICS:ACTION: CALCULATE_METRICSACTION INPUT: - 'TSLA'- 'NVDA'THEN, I WILL ANALYZE THE NEWS SENTIMENT:ACTION: ANALYZE_SENTIMENTACTION INPUT: - 'TSLA'- 'NVDA'AFTER GATHERING ALL THIS DATA, I WILL BE ABLE TO COMPARE THE TWO STOCKS ON VOLATILITY, SHARPE RATIO, AND SENTIMENT. THE STOCK WITH LOWER VOLATILITY, HIGHER SHARPE RATIO (WHICH INDICATES BETTER RISK-ADJUSTED RETURNS), AND POSITIVE SENTIMENT WOULD GENERALLY BE CONSIDERED BETTER FOR A MODERATE RISK PROFILE.HOWEVER, IT'S IMPORTANT TO REMEMBER THAT INVESTING ALWAYS INVOLVES RISKS AND UNCERTAINTIES. THE PAST PERFORMANCE OF A STOCK DOES NOT GUARANTEE ITS FUTURE PERFORMANCE. ALSO, THIS ANALYSIS IS FOR EDUCATIONAL PURPOSES AND SHOULD NOT BE CONSIDERED PROFESSIONAL FINANCIAL ADVICE. ALWAYS CONSULT WITH A FINANCIAL ADVISOR BEFORE MAKING INVESTMENT DECISIONS."

Error fetching - 'TSLA'
- 'NVDA'

Next, I will calculate the financial metrics:

Action: calculate_metrics
Action Input: 
- 'TSLA'
- 'NVDA'

Then, I will analyze the news sentiment:

Action: analyze_sentiment
Action Input: 
- 'TSLA'
- 'NVDA'

After gathering all this data, I will be able to compare the two stocks on volatility, Sharpe ratio, and sentiment. The stock with lower volatility, higher Sharpe ratio (which indicates better risk-adjusted returns), and positive sentiment would generally be considered better for a moderate risk profile.

However, it's important to remember that investing always involves risks and uncertainties. The past performance of a stock does not guarantee its future performance. Also, this analysis is for educational purposes and should not be considered professional financial advice. Always consult with a financial advisor before making investment decisions.: No data found for - 'TSLA'
- 'NVDA'

Next, I will calculate the financial metrics:

Action: calcul

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: - 'TSLA'- 'NVDA'"}}}
ERROR:yfinance:$- 'TSLA'
- 'NVDA': possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


Error fetching - 'TSLA'
- 'NVDA': No data found for - 'TSLA'
- 'NVDA'It seems there is a persistent issue fetching the data for TSLA and NVDA. I will need to troubleshoot this issue before I can proceed with the analysis. In the meantime, I can provide a general comparison based on typical characteristics of these companies. 

Tesla, being a high-growth company in the electric vehicle industry, tends to have higher volatility due to the speculative nature of its business. NVIDIA, on the other hand, is a well-established company in the technology sector with more predictable earnings, which could potentially lead to lower volatility. 

However, without the actual data, it's hard to make a definitive comparison. I recommend trying again later or consulting with a financial advisor for more detailed advice. Remember, investing always involves risks and uncertainties, and this analysis is for educational purposes and should not be considered professional financial advice.Invalid Format: Mi

In [ ]:
# Install required packages
!pip install langchain langchain-openai yfinance pandas numpy matplotlib plotly newsapi-python praw python-dotenv

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 8.9 MB/s eta 0:00:00
